In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph,START,END

In [2]:
## step 1 Define State
class BMIState(TypedDict):
    weight_kg:float
    height_met:float
    bmi:float
    category:str
    


In [3]:
def calculate_bmi(state:BMIState):
    weight=state['weight_kg']
    height=state['height_met']
    bmi=weight/(height**2)
    return {'bmi':bmi}

def calculate_category(state:BMIState):
    bmi=state['bmi']
    
    if bmi < 18.5:
        category = "Underweight"
    elif 18.5 <= bmi < 25:
        category = "Normal"
    elif 25 <= bmi < 30:
        category = "Overweight"
    else:
        category = "Obese"
    return {'category':category}


In [4]:
## Define graph
graph=StateGraph(BMIState)

## define node
graph.add_node('calculate_bmi',calculate_bmi)
graph.add_node('calculate_category',calculate_category)

## add edge
graph.add_edge(START,'calculate_bmi')
graph.add_edge('calculate_bmi','calculate_category')
graph.add_edge('calculate_category',END)


## compile
workflow=graph.compile()

In [10]:
init_state={'weight_kg':1000, 'height_met':1.73}
output=workflow.invoke(init_state)
print(output)

{'weight_kg': 1000, 'height_met': 1.73, 'bmi': 334.1240936883959, 'category': 'Obese'}


## Adding llm

### 1.Example

In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api=os.getenv('groq_api')

In [3]:
llm=ChatGroq(model='gemma2-9b-it',api_key=groq_api)

In [15]:
class LLMState(TypedDict):
    question:str
    answer:str

In [16]:
def llmqa(state:LLMState):
    question=state['question']
    response=llm.invoke(question)
    return {'answer':response}

In [17]:
graph=StateGraph(LLMState)

graph.add_node('llm_qa',llmqa)

graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

workflow=graph.compile()

In [18]:
init_state={'question':'what is captial of bihar'}

workflow.invoke(init_state)

{'question': 'what is captial of bihar',
 'answer': AIMessage(content='The capital of Bihar is **Patna**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_time': 0.023636364, 'prompt_time': 0.00124619, 'queue_time': 2.9912322749999998, 'total_time': 0.024882554}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bc55a537-f04f-4c58-b485-f02fc2d04405-0', usage_metadata={'input_tokens': 16, 'output_tokens': 13, 'total_tokens': 29})}

### 2.Example

In [4]:
class BlogState(TypedDict):
    topic:str
    outline:str
    blog:str

In [5]:
def generate_outline(state:BlogState):
    topic=state['topic']
    prompt=f'Generate a detail outline for a blog on the topic of {topic}'

    response=llm.invoke(prompt)
    return {'outline':response}

def generate_content(state:BlogState):
    topic=state['topic']
    outline=state['outline']
    prompt=f'generate a detail blog on {topic} using this outline \n {outline}'
    content=llm.invoke(prompt)
    return {'blog':content} 

In [7]:
graph=StateGraph(BlogState)

graph.add_node('generate_outline',generate_outline)
graph.add_node('generate_content',generate_content)

graph.add_edge(START,'generate_outline')
graph.add_edge('generate_outline','generate_content')
graph.add_edge('generate_content',END)
workflow=graph.compile()

In [8]:
init_state={'topic':'WWE in India'}
output=workflow.invoke(init_state)
print(output)

{'topic': 'WWE in India', 'outline': AIMessage(content='## WWE in India: From VHS Tapes to Mainstream Mania \n\n**I. Introduction**\n\n* Hook: Anecdote about a personal experience with WWE in India (e.g., watching a VHS tape with friends, the excitement of a live event)\n* Briefly introduce WWE as a global phenomenon\n* Thesis statement:  WWE has evolved from a niche interest to a mainstream entertainment force in India, captivating audiences with its blend of athleticism, storytelling, and larger-than-life characters.\n\n**II. The Early Days: VHS Tapes and Underground Fandom**\n\n* Discuss the introduction of WWE to India through VHS tapes in the 80s and 90s\n* Highlight the impact of legendary wrestlers like Hulk Hogan and The Undertaker on Indian audiences\n*  Describe the emergence of underground fan communities and the creation of fan-made content (e.g., posters, magazines, forums)\n* Mention the challenges of accessing WWE content legally in those days\n\n**III.  The Rise of Tele

In [9]:
print(output['outline'])

content='## WWE in India: From VHS Tapes to Mainstream Mania \n\n**I. Introduction**\n\n* Hook: Anecdote about a personal experience with WWE in India (e.g., watching a VHS tape with friends, the excitement of a live event)\n* Briefly introduce WWE as a global phenomenon\n* Thesis statement:  WWE has evolved from a niche interest to a mainstream entertainment force in India, captivating audiences with its blend of athleticism, storytelling, and larger-than-life characters.\n\n**II. The Early Days: VHS Tapes and Underground Fandom**\n\n* Discuss the introduction of WWE to India through VHS tapes in the 80s and 90s\n* Highlight the impact of legendary wrestlers like Hulk Hogan and The Undertaker on Indian audiences\n*  Describe the emergence of underground fan communities and the creation of fan-made content (e.g., posters, magazines, forums)\n* Mention the challenges of accessing WWE content legally in those days\n\n**III.  The Rise of Television and Cable Access**\n\n* Explain the turn

In [10]:
print(output['blog'])

content='## WWE in India: From VHS Tapes to Mainstream Mania \n\nThe grainy image flickered on the TV screen, sunlight slicing through the dusty window, illuminating a scene of chaos and athleticism. Hulk Hogan, muscles bulging, slammed Andre the Giant with a thunderous body slam. My friends and I, huddled together on the worn floor, erupted in cheers. That afternoon, watching a bootleg VHS tape of WrestleMania III, we weren\'t just watching a wrestling match; we were transported to a world of larger-than-life heroes, epic villains, and electrifying action. This was our introduction to WWE, a global phenomenon that would soon capture the hearts and imaginations of millions in India.\n\nFrom its humble beginnings as a niche interest fueled by smuggled VHS tapes to its current status as a mainstream entertainment force, WWE\'s journey in India is a fascinating story of cultural evolution and fan dedication. WWE has become more than just wrestling; it\'s a cultural touchstone, a source of